# <h2 style= "text-align:center"><b> Đồ án thực hành </b></h2>

# <h1 style='color:red;text-align:center;font-size:32px'><b>Đề tài: Natural Language Processing with Disaster Tweets</b></h1>

# <h1 style='color:blue;text-align:center;font-size:30px'><b>Khám phá dữ liệu + Tiền xử lý</b></h1>

## Import những thư viện cần thiết

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer

### Đọc dữ liệu

In [3]:
data_train=pd.read_csv('train.csv')
data_test=pd.read_csv('test.csv')

In [4]:
# data_train
data_train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# data_test
data_test.head(5)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


## 1. Task 3(Khám phá dữ liệu)

#### 1.1 Shape

+ Tập train có 7613 dòng và 5 cột
+ Tập test có 3263 hàng và 4 cột

In [6]:
data_train.shape

(7613, 5)

In [7]:
data_test.shape

(3263, 4)

#### 1.2 Ý nghĩa mỗi cột

Dữ liệu gồm 5 cột và mỗi cột mạng một ý nghĩa riêng:
+ id: một mã định danh duy nhất cho mỗi tweet
+ text: văn bản của tweet
+ location: vị trí gửi tweet từ đó (có thể để trống)
+ keyword: một từ khóa cụ thể từ tweet (có thể để trống)
+ target: chỉ trong train.csv , điều này biểu thị liệu một tweet có phải là về một thảm họa thực sự (1) hay không ( 0)

In [8]:
data_train.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

#### 1.3 Ý nghĩa các dòng

Mỗi dòng là một tweet dùng để phân loại (dán nhãn)

#### 1.4 Kiểm tra dữ liệu lặp

In [9]:
data_train[data_train.duplicated()].shape[0]

0

In [10]:
data_test[data_test.duplicated()].shape[0]

0

Nhận xét: Cả dữ liệu train và test để không dòng nào bị lặp

#### 1.5 Thông tin tổng quát mỗi cột

In [11]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Nhận xét: Ở tập train cột keyword có 61 ô bị thiếu và ở cột location có 2533 ô bị thiếu

In [12]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


Nhận xét: Ở tập test  cột keyword có 26 ô bị thiếu và ở cột location có 1105 ô bị thiếu

## 2. Task 4(Tiền xử lý dữ liệu)

+ Như chúng ta thấy ở trên cột keyword, location và text đều là văn bản và đều mang ý nghĩa thể hiện xem dòng tweet đó có mang ý nghĩa tích cực hay không. Vì thế ở đây chúng ta sẽ nối 3 cột lại thành một cột là một đoạn văn bản.
+ Ở tập train điều chúng ta chú ý là 3 cột văn bản và cột label nên chúng ta sẽ tạo ra một dataframe mới gồm 2 cột là text và label
+ Ở tập test thì điều chúng ta quan tâm là cột id và cột văn bản nên dataframe của tập test sẽ là 2 cột id và text 

#### 2.1 Fill NA function

In [13]:
#
def make_new_dataframe(data:pd.DataFrame,test=1):
    """
    tạo ra một dataframe mới từ dataframe đầu vào
    
    Parameters:
    - data: dataframe đầu vào
    - test: 1 nếu là tập test, 0 nếu là tập train
    
    Returns:
    - Dataframe mới đã fill các giá trị bị thiếu và các cột đã được nối lại
    """
    data=data.fillna("")
    text=[]
    id=[]
    label=[]
    for index, row in data.iterrows():
        x=str( row["text"]+ " " +row["keyword"]+" " + row["location"])
        text.append(x)
        if test:
            id.append(row['id'])
        else:
            label.append(row['target'])
    if test:
        return {'id':id,'text':text}
    return {'text':text,'label':label}

#### 2.2 Preprocessing text

In [14]:
def preprocess_text(sen):
    """
    Xử lý đoạn văn bản đầu vào.
    
    Parameters:
    - sen: Đoạn văn bản cần xử lý.
    
    Returns:
    - Đoạn văn bản mới sau khi loại bỏ các kí tự không cần thiết.
    """
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    stops = stopwords.words('english')
    #print(stops)
    porter = PorterStemmer()
    for word in sentence.split():
        if word in stops:
            sentence = sentence.replace(word, '')
        sentence = sentence.replace(word, porter.stem(word))
    return sentence.lower()

#### 2.3 Apply in data

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# train_data
train_new=pd.DataFrame(make_new_dataframe(data_train,test=0))
train_new['text']=train_new['text'].apply(preprocess_text)

In [17]:
train_new.head()

,text,label
0,our deed reason earthquak may ah forgiv us,1
1,forest fire near la rong sask canada,1
2,all resid ask shelter place beg notifi off...,1
3,peopl receiv wildfir evacu order california,1
4,just got sent photo rubi alka smoke wildfi...,1


In [18]:
# test_data
test_new=pd.DataFrame(make_new_dataframe(data_test))
test_new['text']=test_new['text'].apply(preprocess_text)

In [19]:
test_new.head(5)

,id,text
0,0,just happen terribl car crash
1,2,heard earthquak differ citi stay safe everyon
2,3,forest fire spot pond gees flee across st...
3,9,apocalyps light spokan wildfir
4,11,typhoon soudelor kill cha taiwan


## 3. Write to new file CSV

In [20]:
# train data
train_new.to_csv('new_train.csv',index=False)

In [21]:
# test data
test_new.to_csv('new_test.csv',index=False)